In [1]:
import pandas as pd

In [2]:
from src.parsers import FontankaPage
from src.parsers.utils import download_url

In [3]:
s = open('./raw/links/links_fontanka.txt').read()

is_link = lambda word: word.startswith('https')
filter_link = lambda row: list(filter(is_link, row.split()))

links = list(map(filter_link, s.splitlines()))
links = list(filter(lambda arr: len(arr), links))
links = [x[0] for x in links]
links[:3]

['https://www.fontanka.ru/2018/01/11/064/',
 'https://www.fontanka.ru/2018/01/12/047/',
 'https://www.fontanka.ru/2018/01/13/052/']

In [4]:
from time import sleep
from tqdm import tqdm

records = []

for i, link in tqdm(enumerate(links), total=len(links)):
    sleep(0.2)
    
    try:
        source = download_url(link)
    except Exception as e:
        print(f'error downloading page {i} with err {e}')
        continue
    
    page_name = link.split('/')[-2]
    with open(f'raw/pages/{page_name}', 'w') as f:
        f.write(source)
    
    try:
        content = FontankaPage(source)
        records.append(content)
    except Exception as e:
        print(f'error parsing page {i} with err {e}')
        continue

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 803/803 [09:25<00:00,  1.42it/s]


In [5]:
def make_tuple(p) -> tuple:
    try: 
        return (p.get_header(), p.get_main_text(), p.get_create_date()) 
    except: 
        print('except')
        return ('header', 'text', '2022-01-01')

tuples = [make_tuple(page) for page in records]

df = pd.DataFrame.from_records(tuples, columns=['title', 'text', 'create_dttm']).assign(link=links)
print(df.shape)
df.head()

except
(803, 4)


,title,text,create_dttm,link
0,Люди без подписи,Автор петиции «Прекратите ущемление прав людей...,"11 января 2018, 13:41",https://www.fontanka.ru/2018/01/11/064/
1,В Петербурге инвалидам хотят разрешить ездить ...,Депутаты петербургского Законодательного собра...,"12 января 2018, 11:55",https://www.fontanka.ru/2018/01/12/047/
2,Дана хочет слышать,"«Хорошо учится, любит читать…», - слова мамы о...","13 января 2018, 18:05",https://www.fontanka.ru/2018/01/13/052/
3,Волонтёры ГАООРДИ подарили праздник более 200 ...,Санкт-Петербургская ассоциация общественных об...,"15 января 2018, 10:45",https://www.fontanka.ru/2018/01/15/026/
4,Из-за отсутствия социального такси сосновоборс...,Прокурор города Сосновый Бор Ленинградской обл...,"16 января 2018, 13:01",https://www.fontanka.ru/2018/01/16/924/


In [6]:
df.to_csv('./raw/articles/fontanka.csv', index=False)